# **Description**
---

1. Suppose we have 1 M USD and want to invest in US stock market for 12 years (2012-2024). We will decide the portfolio every three months and buy hold for three months.  

  - At the beginning of every three months: Use the past six months of data to compute correlations of S&P 500 stocks based on stock returns.

  - Select stocks for the portfolio as follows:
    - Choose the first stock with the smallest average of the absolute values of correlations with all other stocks (based on returns).

    - Choose the second stock with the smallest absolute correlation with the first selected stock's return.
    - Choose the third stock with the smallest average of absolute correlations with the returns of the first two selected stocks.
  -  Repeat this process until we have selected a total of 10 stocks.
  
  - Invest an equal amount in each of these 10 stocks, and hold them for three months.
    
  - Repeat this selection and investment process every three months over the 12-year period.


2. We calculate the maximum drawdown and annual Sharpe ratio of our investment over these 12 years.  

  - Track the total wealth of our portfolio daily, and use this wealth process to determine the maximum drawdown and daily Sharpe ratio. Then, convert the daily Sharpe ratio to an annual Sharpe ratio.


3. Conclusion and results comparison with market index ETFs: QQQ and SPY.

4. Further Strategy analysis

A place to share and store our code.


https://docs.google.com/document/d/1RyAavOpPLeXK95ag0hyoUD-47i3UcIJYSJd6b6iSZMs/edit?usp=sharing



In [1]:
# @title 1. !pip Install
!pip install --quiet --upgrade rmbi3110
!pip install --quiet --upgrade dcor
!pip install yfinance

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.0 MB/s eta 0:00:00


In [3]:
# @title 2. Library Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings
from plotly.subplots import make_subplots
warnings.filterwarnings('ignore')
pd.options.plotting.backend = "plotly"
import yfinance as yf
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sns
import numpy as np
import rmbi3110
import dcor

In [4]:
# @title Extract Data and create DataFrame (can omit this line of code as it would performed below)
adf = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
  # Extract "Company Name" Data
sp_500_list = adf['Symbol'].tolist()
sp_500_list
  # Create List
index=sp_500_list.index("BRK.B")
sp_500_list[index]="BRK-B"
index=sp_500_list.index("BF.B")
sp_500_list[index]="BF-B"
  # Debug

Return_DataFrame=pd.DataFrame({})
  # Create Empty DataFrame for Return

In [ ]:
# @title (Old) Default code
start_year=2011
start_month=4
end_year=2011
end_month=9
end_date=31
Start_Date=f"{start_year}-{start_month}-01"
End_Date=f"{end_year}-{end_month}-{end_date}"

selected_stocks = []

for trial in range(48):
  start_year=2011
  start_year=start_year+(trial+2)//4
  if (trial+2)%4==0:
    start_month=start_month-9
  else:
    start_month=start_month+3
  end_year=2011
  end_year=end_year+(trial+3)//4
  if (trial+3)%4==0:
    end_month=end_month-9
  else:
    end_month=end_month+3
  if end_month == 3 or end_month == 12:
    end_date=31
  else:
    end_date=30

  Start_Date=f"{start_year}-{start_month}-01"
  End_Date=f"{end_year}-{end_month}-{end_date}"

  for i in range(len(sp_500_list)):
    k=yf.Ticker(sp_500_list[i]).history(start=Start_Date, end=End_Date)
    Return_DataFrame[sp_500_list[i]]=(k['Close']/k['Open']-1)

  # Add Return to DataFrame for each stock
  Return_DataFrame=Return_DataFrame.fillna(0)

  Correlation_DataFrame=Return_DataFrame.corr()
  # Correlation of Stocks

  Abs_Correlation_DataFrame=abs(Correlation_DataFrame)
  # Absolute Value of Correlation of Stocks

### The following was generated by Deepseek ###

# Number of stocks to select
  num_stocks = 10

# Loop to select stocks
  for k in range(num_stocks):
    if k == 0:
        # Select the first stock
      selected_stocks.append(Abs_Correlation_DataFrame.mean().sort_values(ascending=True).index[0])
    else:
        # Calculate the cumulative sum of correlations for the selected stocks
      cumulative_corr = sum(Abs_Correlation_DataFrame[stock] for stock in selected_stocks)
        # Select the next stock
        # Avoid Repetition
      j=0
      while cumulative_corr.sort_values().index[j] in selected_stocks:
        j=j+1
      selected_stocks.append(cumulative_corr.sort_values().index[j])

  print(selected_stocks)


ERROR:yfinance:$ABBV: possibly delisted; no price data found  (1d 2011-7-01 -> 2011-12-31) (Yahoo error = "Data doesn't exist for startDate = 1309492800, endDate = 1325307600")
ERROR:yfinance:$ABNB: possibly delisted; no price data found  (1d 2011-7-01 -> 2011-12-31) (Yahoo error = "Data doesn't exist for startDate = 1309492800, endDate = 1325307600")
Exception ignored from cffi callback <function buffer_callback at 0x7b1a756f1620>:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/curl_cffi/curl.py", line 68, in buffer_callback
    @ffi.def_extern()
    
KeyboardInterrupt: 
ERROR:yfinance:$AKAM: possibly delisted; no price data found  (1d 2011-7-01 -> 2011-12-31)
ERROR:yfinance:$ALLE: possibly delisted; no price data found  (1d 2011-7-01 -> 2011-12-31) (Yahoo error = "Data doesn't exist for startDate = 1309492800, endDate = 1325307600")
Exception ignored from cffi callback <function buffer_callback at 0x7b1a756f1620>:
Traceback (most recent call last):

In [ ]:
# @title Get Return From Data (can omit this line of code as it would performed below)
for i in range(len(sp_500_list)):
  k=yf.Ticker(sp_500_list[i]).history(start="2011-07-01", end="2025-01-31")
  Return_DataFrame[sp_500_list[i]]=(k['Close']/k['Open']-1)

In [6]:
import time

t1 = time.time()

# @title Main Code (first part), get 10 stocks each for all 48 periods + calculate the daily return + Maximum Drawdown
adf = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
  # Extract "Company Name" Data
sp_500_list = adf['Symbol'].tolist()
sp_500_list
  # Create List
index=sp_500_list.index("BRK.B")
sp_500_list[index]="BRK-B"
index=sp_500_list.index("BF.B")
sp_500_list[index]="BF-B"
  # Debug
#for i in range(len(sp_500_list)):
kkk=yf.Ticker("SPY").history(start="2011-07-01", end="2025-1-31")
Return_DataFrame=pd.DataFrame(index=kkk.index.tolist())
#lll=yf.Ticker("SPY").history(start="2012-01-01", end="2023-12-31")
#Portfolio_Return=pd.DataFrame(index=lll.index.tolist()+["2023-12-31"])
  # Create DataFrame with Date for Return
balance_list = [1000000,]
date_list = ["2012-01-01",]
balance_df = 0
investment_capital = 1000000
investment_capital_per_stock = []
for n in range(10):
  investment_capital_per_stock.append(investment_capital / 10)

start_year=2011
start_month=4
end_year=2011
end_month=9
invest_start_month=10
invest_end_month=12
#Start_Date=f"{start_year}-{start_month}-01"# (remove)
#End_Date=f"{end_year}-{end_month}-01"# (remove)
selected_stocks = []
selected_stocks_infinite = []
  # store stocks for investment (480 stocks in total)
Period_date_total = []
  # number of days per priod (48 numbers in total)

for i in range(len(sp_500_list)):
  k=yf.Ticker(sp_500_list[i]).history(start="2011-07-01", end="2025-1-31").fillna(0)
  stock_data = []
  for r in range(len(k.index)-1):
    r1 = k["Open"][r+1]
    r2 = k["Open"][r]
    r3 = ((r1/r2) -1)
    stock_data.append(r3)
  stock_data.append(0)
  if len(stock_data)-len(Return_DataFrame) != 0:
    for s in range(len(stock_data)-len(Return_DataFrame)):
      stock_data = [0] + stock_data
  else:
    Return_DataFrame[sp_500_list[i]]=stock_data
  # GET RETURN % for stocks (Return at day x = x+1[open]-x[open])

  #t=yf.Ticker(sp_500_list[i]).history(start="2011-07-01", end="2025-1-31").fillna(0)
  #Return_DataFrame[sp_500_list[i]]=(t["Open"].shift(-1)-t["Open"]).iloc[:-1]


  #k1 = k[1:]['Open'].reset_index(drop=True)
  #k2 = k[:-1]['Open'].reset_index(drop=True)
  #k3 = (k1 / k2)-1
  #Return_DataFrame[sp_500_list[i]] = (k1 / k2)-1
  #for q in range(len(k)-1):
   # Return_DataFrame[sp_500_list[i]]=(k['Open'].iloc(q+1)/k['Open'].iloc(q)-1)
    # Get data of stock return

for trial in range(48):
  start_year=2011
  start_year=start_year+(trial+2)//4
  if (trial+2)%4==0:
    start_month=start_month-9
  else:
    start_month=start_month+3
  end_year=2011
  end_year=end_year+(trial+3)//4
  if (trial+3)%4==0:
    end_month=end_month-9
  else:
    end_month=end_month+3



  invest_start_year = 2012
  invest_start_year = invest_start_year+(trial)//4

  if (trial)%4==0:
    invest_start_month = invest_start_month-9
  else:
    invest_start_month =invest_start_month+3

  invest_end_year = 2012
  invest_end_year=invest_end_year+(trial)//4

  if (trial)%4==0:
    invest_end_month=invest_end_month-9
  else:
    invest_end_month=invest_end_month+3

  if invest_end_month == 3 or invest_end_month == 12:
    invest_end_day = 31
  else:
    invest_end_day = 30

  if end_month == 3 or end_month == 12:
    end_day = 31
  else:
    end_day = 30



  Invest_Start_Date = f"{invest_start_year}-{invest_start_month}-01"
  Invest_End_Date = f"{invest_end_year}-{invest_end_month}-{invest_end_day}"

  Start_Date=f"{start_year}-{start_month}-01"
  End_Date=f"{end_year}-{end_month}-{end_day}"

  print([Start_Date,End_Date,Invest_Start_Date,Invest_End_Date])


  Return_DataFrame_Period=Return_DataFrame.loc[Start_Date:End_Date].fillna(0)


  Correlation_DataFrame_Period=Return_DataFrame_Period.corr()
  # Correlation of Stocks

  Abs_Correlation_DataFrame_Period=abs(Correlation_DataFrame_Period)

  selected_stocks=[]

# Number of stocks to select
  num_stocks = 10

# Loop to select stocks
  for m in range(num_stocks):
    if m == 0:
        # Select the first stock
      selected_stocks.append(Abs_Correlation_DataFrame_Period.mean().sort_values(ascending=True).index[0])
    else:
        # Calculate the cumulative sum of correlations for the selected stocks
      cumulative_corr = sum(Abs_Correlation_DataFrame_Period[stock] for stock in selected_stocks)
        # Select the next stock
        # Avoid Repetition
      j=0
      while cumulative_corr.sort_values(ascending=True).index[j] in selected_stocks:
        j=j+1
      selected_stocks.append(cumulative_corr.sort_values().index[j])

  Return_DataFrame_Period_Invest = Return_DataFrame.loc[Invest_Start_Date:Invest_End_Date].fillna(0)[selected_stocks]

  for l in selected_stocks:
    selected_stocks_infinite.append(l)


  for zz in range(len(Return_DataFrame_Period_Invest)):
    for z in range(len(selected_stocks)):
      investment_capital_per_stock[z]=investment_capital_per_stock[z]*(1+Return_DataFrame_Period_Invest[selected_stocks[z]].iloc[zz])
    #print(sum(investment_capital_per_stock),end=" ")
    balance_list.append(sum(investment_capital_per_stock))
    date_list.append(Return_DataFrame_Period_Invest.index[zz])
  investment_capital = sum(investment_capital_per_stock)

  investment_capital_per_stock = []
  for o in range(10):
    investment_capital_per_stock.append(investment_capital / 10)

balance_df = pd.DataFrame(balance_list,index = date_list,columns=["Balance"])
SPY=yf.Ticker("SPY").history(start="2012-01-01", end="2024-1-3")
SPY_Return = ([0]+((SPY["Open"].shift(-1)/SPY["Open"]-1)*100).fillna(0).tolist())[:-1]
QQQ=yf.Ticker("QQQ").history(start="2012-01-01", end="2024-1-3")
QQQ_Return = ([0]+((QQQ["Open"].shift(-1)/QQQ["Open"]-1)*100).fillna(0).tolist())[:-1]
balance_compare = pd.DataFrame({'Portfolio Balance': balance_list,'SPY value':(SPY["Open"]*1000000/SPY["Open"].iloc[0]).shift(-1),'QQQ value':(QQQ["Open"]*1000000/QQQ["Open"].iloc[0]).shift(-1)},index=date_list).fillna(1000000)

print(investment_capital)
print(selected_stocks_infinite)
print(balance_df)

df_balance_list = pd.DataFrame(balance_list, columns=['Balance'])
Portfolio_Return_list = ((df_balance_list.div(df_balance_list.shift(periods=1).fillna(1000000))-1)*100).rename(columns={"Balance": "Return%"}).set_axis(date_list)
Portfolio_Return_Compare = pd.DataFrame({
    'Portfolio Return %': Portfolio_Return_list['Return%'].to_list(),
    'SPY Return %': SPY_Return,
    'QQQ Return %': QQQ_Return
}, index=date_list)

'''Portfolio_Return_list = [0]
for s in range(len(balance_list)-1):
  s1 = balance_list[s+1]
  s2 = balance_list[s]
  s3 = (((s1/s2) -1)*100)
  Portfolio_Return_list.append(s3)

Portfolio_Return=pd.DataFrame(Portfolio_Return_list,index = date_list,columns=["Return %"])

#Portfolio_Return_Compare = pd.DataFrame([Portfolio_Return_list,SPY_Return],index = date_list,columns=["Return %"])
Portfolio_Return_Compare = pd.DataFrame({
    'Portfolio Return %': Portfolio_Return_list,
    'SPY Return %': SPY_Return,
    'QQQ Return %': QQQ_Return
}, index=date_list)
print(Portfolio_Return)'''

#Maximum Drawdown
#print("The Maximum Drawdown is" , str(min(Portfolio_Return_list)) +"%", "on date" , pd.to_datetime(Portfolio_Return.loc[Portfolio_Return['Return %'] == min(Portfolio_Return_list)].index).strftime('%Y-%m-%d')[0] )

t2 = time.time()

time_run = t2-t1

['2011-7-01', '2011-12-31', '2012-1-01', '2012-3-31']
['2011-10-01', '2012-3-31', '2012-4-01', '2012-6-30']
['2012-1-01', '2012-6-30', '2012-7-01', '2012-9-30']
['2012-4-01', '2012-9-30', '2012-10-01', '2012-12-31']
['2012-7-01', '2012-12-31', '2013-1-01', '2013-3-31']
['2012-10-01', '2013-3-31', '2013-4-01', '2013-6-30']
['2013-1-01', '2013-6-30', '2013-7-01', '2013-9-30']
['2013-4-01', '2013-9-30', '2013-10-01', '2013-12-31']
['2013-7-01', '2013-12-31', '2014-1-01', '2014-3-31']
['2013-10-01', '2014-3-31', '2014-4-01', '2014-6-30']
['2014-1-01', '2014-6-30', '2014-7-01', '2014-9-30']
['2014-4-01', '2014-9-30', '2014-10-01', '2014-12-31']
['2014-7-01', '2014-12-31', '2015-1-01', '2015-3-31']
['2014-10-01', '2015-3-31', '2015-4-01', '2015-6-30']
['2015-1-01', '2015-6-30', '2015-7-01', '2015-9-30']
['2015-4-01', '2015-9-30', '2015-10-01', '2015-12-31']
['2015-7-01', '2015-12-31', '2016-1-01', '2016-3-31']
['2015-10-01', '2016-3-31', '2016-4-01', '2016-6-30']
['2016-1-01', '2016-6-30', '

sharpe ratio

In [ ]:
#version1
daily_return=df_balance_list - df_balance_list.shift(periods=1).fillna(1000000)
print(daily_return)
overall_std_return = daily_return.std()
print(overall_std_return)
sharpe_ratio= daily_return.mean().div(overall_std_return)
print(sharpe_ratio)
annual_sharpe_ratio= sharpe_ratio*np.sqrt(252)
print(annual_sharpe_ratio)

#version2
sharpe_ratio2=Portfolio_Return_list.mean().div(Portfolio_Return_list.std())
print(sharpe_ratio2)
annual_sharpe_ratio2= sharpe_ratio2*np.sqrt(252)
print(annual_sharpe_ratio2)

In [ ]:
# @title Line Plot (Portfolio Return %)
Portfolio_Return_list.plot.line()

In [ ]:
# @title Line Plot (Balance)
balance_df.plot.line()

In [ ]:
# @title Line Plot (% Compare)
Portfolio_Return_Compare.plot.line()

In [ ]:
# @title Line Plot (Return Compare)
balance_compare.plot.line()

In [ ]:
# @title Maximum drawdown
cum_max=Portfolio_Return_list.cummax()
drawdown=cum_max-Portfolio_Return_list
max_drawdown=drawdown.max()
max_drawdown

,0
Return%,16.617444


In [ ]:
Portfolio_Return_list = [0]
for s in range(len(balance_df)-1):
  s1 = balance_df.iloc[s+1]
  s2 = balance_df.iloc[s]
  s3 = ((s1/s2) -1)
  Portfolio_Return_list.append(s3)
#Portfolio_Return_list.append(0)
Portfolio_Return = balance_df = pd.DataFrame(Portfolio_Return_list,index = date_list,columns=["Return %"])
print(Portfolio_Return)

                                                       Return %
2012-01-01                                                    0
2012-01-03 00:00:00-05:00        Return %    inf
dtype: float64
2012-01-04 00:00:00-05:00        Return %   -1.0
dtype: float64
2012-01-05 00:00:00-05:00   Return %   -1.018336
dtype: float64
2012-01-06 00:00:00-05:00  Return %   -32.489081
dtype: float64
...                                                         ...
2023-12-22 00:00:00-05:00   Return %    0.002912
dtype: float64
2023-12-26 00:00:00-05:00   Return %   -0.736699
dtype: float64
2023-12-27 00:00:00-05:00    Return %    2.72328
dtype: float64
2023-12-28 00:00:00-05:00   Return %   -4.958544
dtype: float64
2023-12-29 00:00:00-05:00   Return %   -1.580259
dtype: float64

[3043 rows x 1 columns]


In [ ]:
balance_df

,Balance
2012-01-01,1.000000e+06
2012-01-03 00:00:00-05:00,1.000587e+06
2012-01-04 00:00:00-05:00,1.004650e+06
2012-01-05 00:00:00-05:00,1.008287e+06
2012-01-06 00:00:00-05:00,1.024424e+06
...,...
2023-12-22 00:00:00-05:00,8.042436e+06
2023-12-26 00:00:00-05:00,8.038546e+06
2023-12-27 00:00:00-05:00,8.039536e+06
2023-12-28 00:00:00-05:00,8.045445e+06


In [ ]:
len(k.index)

3228

In [ ]:
yf.Ticker("PLTR").history(start="2011-07-01", end="2025-1-31").fillna(0)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-09-30 00:00:00-04:00,10.000000,11.410000,9.110000,9.500000,338584400,0.0,0.0
2020-10-01 00:00:00-04:00,9.690000,10.100000,9.230000,9.460000,124297600,0.0,0.0
2020-10-02 00:00:00-04:00,9.060000,9.280000,8.940000,9.200000,55018300,0.0,0.0
2020-10-05 00:00:00-04:00,9.430000,9.490000,8.920000,9.030000,36316900,0.0,0.0
2020-10-06 00:00:00-04:00,9.040000,10.180000,8.900000,9.900000,90864000,0.0,0.0
...,...,...,...,...,...,...,...
2025-01-24 00:00:00-05:00,79.050003,82.239998,78.440002,78.980003,77931900,0.0,0.0
2025-01-27 00:00:00-05:00,74.070000,76.720001,72.669998,75.440002,76093300,0.0,0.0
2025-01-28 00:00:00-05:00,75.540001,80.739998,74.040001,80.230003,65916700,0.0,0.0


In [ ]:
k.index.tolist()

[Timestamp('2013-02-01 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-04 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-05 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-06 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-07 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-08 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-11 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-12 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-13 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-14 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-15 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-19 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-20 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-21 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-22 00:00:00-0500', tz='America/New_York'),
 Timestamp('2013-02-25 00:00:00-0500', tz='America/New_

AttributeError: 'function' object has no attribute 'index'

In [ ]:
k

<bound method Index.drop of DatetimeIndex(['2011-07-01 00:00:00-04:00', '2011-07-05 00:00:00-04:00',
               '2011-07-06 00:00:00-04:00', '2011-07-07 00:00:00-04:00',
               '2011-07-08 00:00:00-04:00', '2011-07-11 00:00:00-04:00',
               '2011-07-12 00:00:00-04:00', '2011-07-13 00:00:00-04:00',
               '2011-07-14 00:00:00-04:00', '2011-07-15 00:00:00-04:00',
               ...
               '2025-01-16 00:00:00-05:00', '2025-01-17 00:00:00-05:00',
               '2025-01-21 00:00:00-05:00', '2025-01-22 00:00:00-05:00',
               '2025-01-23 00:00:00-05:00', '2025-01-24 00:00:00-05:00',
               '2025-01-27 00:00:00-05:00', '2025-01-28 00:00:00-05:00',
               '2025-01-29 00:00:00-05:00', '2025-01-30 00:00:00-05:00'],
              dtype='datetime64[ns, America/New_York]', name='Date', length=3416, freq=None)>

In [ ]:
Correlation_DataFrame_Period

,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,...,WMB,WTW,WDAY,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS
MMM,1.000000,0.394485,0.168608,0.165166,0.187710,0.147794,-0.078481,0.349428,0.415547,0.005364,...,0.409494,0.176913,0.040307,0.233366,0.114092,0.450314,0.156546,0.245888,0.256275,0.240347
AOS,0.394485,1.000000,-0.049467,0.094779,0.170712,0.169374,0.089455,0.112927,0.237141,0.097593,...,0.219348,0.199568,0.169587,0.349483,-0.043711,0.447829,0.140545,0.390091,0.166211,0.211752
ABT,0.168608,-0.049467,1.000000,0.254850,0.021884,0.069264,-0.012896,0.113565,0.124774,0.237371,...,-0.000233,0.233264,0.187780,0.116968,0.309636,0.252656,0.277414,0.057415,0.458965,0.247503
ABBV,0.165166,0.094779,0.254850,1.000000,0.022223,-0.118914,-0.255016,0.171826,0.102475,0.076387,...,0.171917,0.306629,-0.108609,0.083361,0.256606,0.101445,0.129265,-0.137068,0.205894,0.062498
ACN,0.187710,0.170712,0.021884,0.022223,1.000000,0.457422,0.251783,0.300914,0.196708,0.209442,...,0.133505,0.191764,0.370665,0.255253,0.060785,0.056944,0.089064,0.252029,0.159494,0.061345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL,0.450314,0.447829,0.252656,0.101445,0.056944,0.147487,0.089752,0.339919,0.374815,0.057211,...,0.288068,0.199833,0.215638,0.337202,0.141826,1.000000,0.327315,0.328388,0.375193,0.384065
YUM,0.156546,0.140545,0.277414,0.129265,0.089064,0.157131,-0.010258,0.142816,0.330139,0.219377,...,0.244858,0.247032,0.195550,0.249591,0.157151,0.327315,1.000000,0.194563,0.206953,0.333772
ZBRA,0.245888,0.390091,0.057415,-0.137068,0.252029,0.227632,0.382963,0.073504,0.117565,0.065191,...,0.357883,0.183041,0.347481,0.321729,-0.002737,0.328388,0.194563,1.000000,0.203128,0.307774
ZBH,0.256275,0.166211,0.458965,0.205894,0.159494,0.220785,0.016789,0.184907,0.181596,0.110563,...,0.155957,0.348124,0.252137,0.229976,0.311782,0.375193,0.206953,0.203128,1.000000,0.149903


In [ ]:
test123=[]
for ss in range(len(abcde)):
  if abcde[ss] != selected_stocks_infinite[ss]:
    test123.append("error")
  else:
    test123.append("true")
test123.count("error")

63

In [ ]:
date_list

[Timestamp('2012-01-03 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-04 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-05 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-06 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-09 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-10 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-11 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-12 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-13 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-17 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-18 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-19 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-20 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-23 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-24 00:00:00-0500', tz='America/New_York'),
 Timestamp('2012-01-25 00:00:00-0500', tz='America/New_

In [ ]:
balance_df

,Balance
2011-01-01,1.000000e+06
2012-01-03 00:00:00-05:00,9.998692e+05
2012-01-04 00:00:00-05:00,1.015715e+06
2012-01-05 00:00:00-05:00,1.018423e+06
2012-01-06 00:00:00-05:00,1.031117e+06
...,...
2023-12-22 00:00:00-05:00,5.722952e+06
2023-12-26 00:00:00-05:00,5.747205e+06
2023-12-27 00:00:00-05:00,5.780560e+06
2023-12-28 00:00:00-05:00,5.757270e+06


In [ ]:
investment_capital/1000000

np.float64(150332481368607.56)

In [ ]:
len(Return_DataFrame["XYL"].loc['2012-1-02':'2012-1-07'])

4

In [ ]:
for z in range(len(selected_stocks)):
  for zz in range(len(Return_DataFrame_Period_Invest)):
      investment_capital_per_stock[z]=investment_capital_per_stock[z]*(1+Return_DataFrame_Period_Invest[selected_stocks[z]].iloc[zz])
print(investment_capital_per_stock)

[np.float64(100508.77233210995), np.float64(99460.63349822724), np.float64(104041.64014047582), np.float64(111686.81112907975), np.float64(110453.89724171702), np.float64(90430.33771775258), np.float64(112695.72083266228), np.float64(112619.64319510762), np.float64(104631.99354265652), np.float64(109447.47102397594)]


In [ ]:
sum(investment_capital_per_stock)

np.float64(1055976.9206537646)

In [ ]:
selected_stocks_infinite

['XYL',
 'TDY',
 'TPL',
 'APTV',
 'NFLX',
 'SO',
 'NEM',
 'HII',
 'REGN',
 'APO',
 'EPAM',
 'RMD',
 'HSY',
 'NFLX',
 'APTV',
 'CL',
 'CBOE',
 'XYL',
 'KMI',
 'TPL',
 'META',
 'MDLZ',
 'CMG',
 'ELV',
 'AXON',
 'TPL',
 'TMUS',
 'VZ',
 'KR',
 'MRK',
 'NOW',
 'WY',
 'DUK',
 'VRTX',
 'META',
 'PANW',
 'PSX',
 'EPAM',
 'CAH',
 'CHD',
 'MNST',
 'ESS',
 'SHW',
 'AXON',
 'BG',
 'UAL',
 'DG',
 'META',
 'MPWR',
 'GILD',
 'AMCR',
 'PPL',
 'NFLX',
 'HCA',
 'UDR',
 'MNST',
 'TMUS',
 'FANG',
 'PM',
 'TPL',
 'AMCR',
 'PCG',
 'WDAY',
 'HCA',
 'BBY',
 'STZ',
 'APA',
 'TPL',
 'CTSH',
 'NFLX',
 'NWSA',
 'COST',
 'AMCR',
 'JNPR',
 'NOW',
 'FCX',
 'CTSH',
 'LYV',
 'CHTR',
 'LH',
 'ALLE',
 'CCL',
 'NRG',
 'LDOS',
 'TSLA',
 'NWS',
 'DE',
 'CDW',
 'HPQ',
 'HUM',
 'AMCR',
 'META',
 'AEP',
 'DVA',
 'HOLX',
 'JNPR',
 'NCLH',
 'HLT',
 'TPL',
 'ZTS',
 'AMCR',
 'STZ',
 'PAYC',
 'EXC',
 'LULU',
 'K',
 'CHTR',
 'GEN',
 'MRK',
 'TPL',
 'SYF',
 'ZBH',
 'EXC',
 'ODFL',
 'AMCR',
 'TRGP',
 'TPL',
 'TECH',
 'DECK',
 'NVR',


In [ ]:
len(selected_stocks)

480

In [ ]:
investment_capital_per_stock[0]

np.float64(100508.77233210995)

In [ ]:
selected_stocks

[]

In [ ]:
for i in range(2, 11):
    globals()[f"stock{i}"]



In [ ]:
Abs_Correlation_DataFrame[Abs_Correlation_DataFrame.mean().dropna().sort_values(ascending=True).index[0]].sort_values()

,XYL
MCHP,0.000087
ACGL,0.000232
LEN,0.000454
JBL,0.000625
FAST,0.000675
...,...
VLTO,NaN
VICI,NaN
VST,NaN
WDAY,NaN


In [ ]:
Abs_Correlation_DataFrame.mean().dropna().sort_values(ascending=True).index[0]

'XYL'

In [ ]:
Correlation_DataFrame=Return_DataFrame.corr()
  # Correlation of Stocks

In [ ]:
Abs_Correlation_DataFrame=abs(Correlation_DataFrame)
  # Absolute Value of Correlation of Stocks

In [ ]:
Abs_Correlation_DataFrame

,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,...,WMB,WTW,WDAY,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS
MMM,1.000000,0.470164,0.385417,0.303936,0.421030,0.299602,0.211045,0.260422,0.375031,0.388421,...,0.220762,0.301570,0.206666,0.231745,0.264264,0.519868,0.341697,0.347992,0.359953,0.285770
AOS,0.470164,1.000000,0.270904,0.235265,0.360721,0.308851,0.245276,0.245048,0.320374,0.394660,...,0.230314,0.259477,0.250691,0.292423,0.148872,0.499547,0.308762,0.370317,0.272915,0.266119
ABT,0.385417,0.270904,1.000000,0.388333,0.416542,0.363721,0.229172,0.175069,0.287917,0.493136,...,0.152327,0.321243,0.279682,0.149510,0.312594,0.375036,0.321985,0.320591,0.417625,0.447177
ABBV,0.303936,0.235265,0.388333,1.000000,0.284088,0.249647,0.129579,0.178619,0.255256,0.336739,...,0.233057,0.211292,0.198491,0.171913,0.197744,0.253012,0.244983,0.193711,0.321655,0.339494
ACN,0.421030,0.360721,0.416542,0.284088,1.000000,0.547341,0.310708,0.300101,0.334690,0.479049,...,0.221350,0.360031,0.416382,0.263505,0.239275,0.429743,0.368972,0.391576,0.375002,0.401014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL,0.519868,0.499547,0.375036,0.253012,0.429743,0.354047,0.277694,0.316627,0.403248,0.446126,...,0.253880,0.328364,0.284270,0.286473,0.242252,1.000000,0.340045,0.388451,0.348912,0.345035
YUM,0.341697,0.308762,0.321985,0.244983,0.368972,0.298290,0.209998,0.276995,0.352570,0.345171,...,0.207640,0.303760,0.280629,0.265834,0.248332,0.340045,1.000000,0.266445,0.348309,0.328568
ZBRA,0.347992,0.370317,0.320591,0.193711,0.391576,0.392001,0.334060,0.257313,0.233909,0.420609,...,0.236266,0.223919,0.339010,0.290475,0.074222,0.388451,0.266445,1.000000,0.274262,0.333922
ZBH,0.359953,0.272915,0.417625,0.321655,0.375002,0.303111,0.199708,0.304228,0.352022,0.387174,...,0.264024,0.276764,0.288285,0.309826,0.173713,0.348912,0.348309,0.274262,1.000000,0.316595


In [ ]:
Abs_Correlation_DataFrame.mean().sort_values(ascending=True)

,0
NEM,0.113098
MRNA,0.119335
PCG,0.123463
SOLV,0.134248
KR,0.143492
...,...
BRK-B,0.373100
AME,0.373354
BLK,0.378070
HON,0.382637


In [ ]:
Abs_Correlation_DataFrame['AMP'].sort_values(ascending=True)

,AMP
PRU,0.731253
RJF,0.711224
PFG,0.704620
MET,0.704601
MS,0.665219
...,...
MRNA,0.135389
KVUE,0.122578
NEM,0.075172
SOLV,0.073992
